In [ ]:
# Import necessary libraries

# Images
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

# PDF's
import fitz

In [ ]:
# Define paths
input_dir = Path("photo/images")
output_dir_mtcnn = Path("face_detected_MTCNN")
cropped_dir_mtcnn = Path("cropped_face_MTCNN")

output_dir_mtcnn.mkdir(parents=True, exist_ok=True)
cropped_dir_mtcnn.mkdir(parents=True, exist_ok=True)

In [ ]:
detector = MTCNN()

no_faces_detected_mtcnn = []

In [ ]:
# def detect_faces_mtcnn(img_path):
#     img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
#     results = detector.detect_faces(img)

#     for idx, res in enumerate(results, start=1):
#         x, y, width, height = res['box']
#         cv2.rectangle(img, (x, y), (x+width, y+height), (255, 0, 0), 2)

#         # Crop and save each face
#         face_crop = img[y:y+height, x:x+width]
#         crop_name = f"{img_path.stem}_face_{idx}.jpg"
#         face_bgr = cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR)
#         cv2.imwrite(str(cropped_dir_mtcnn / crop_name), face_bgr)

#     # Save image with rectangles
#     img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#     output_path = output_dir_mtcnn / img_path.name
#     cv2.imwrite(str(output_path), img_bgr)

#     print(f"{img_path.name}: {len(results)} face(s) detected and cropped (MTCNN).")

# def detect_faces_mtcnn(img_path):
#     img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
#     results = detector.detect_faces(img)

#     if len(results) == 0:
#         no_faces_detected_mtcnn.append(img_path)
#     else:
#         for idx, res in enumerate(results, start=1):
#             x, y, width, height = res['box']
#             cv2.rectangle(img, (x, y), (x+width, y+height), (255, 0, 0), 2)

#             # Crop and save each face
#             face_crop = img[y:y+height, x:x+width]
#             crop_name = f"{img_path.stem}_face_{idx}.jpg"
#             face_bgr = cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR)
#             cv2.imwrite(str(cropped_dir_mtcnn / crop_name), face_bgr)

#     # Save image with rectangles
#     img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(str(output_dir_mtcnn / img_path.name), img_bgr)
#     print(f"MTCNN processed: {img_path.name}, faces detected: {len(results)}")

def detect_faces_mtcnn(img_path, is_pdf=False):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Error: {img_path}")
        return
        
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)

    if len(results) == 0:
        no_faces_detected_mtcnn.append(img_path)
    else:
        for idx, res in enumerate(results, start=1):
            x, y, width, height = res['box']
            x, y = max(0, x), max(0, y)
            
            if not is_pdf:  # Solo dibujar rectángulo si no es PDF
                cv2.rectangle(img_rgb, (x, y), (x+width, y+height), (255, 0, 0), 2)
            
            face_crop = img_rgb[y:y+height, x:x+width]
            crop_name = f"{img_path.stem}_face_{idx}.jpg"
            face_bgr = cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(cropped_dir_mtcnn / crop_name), face_bgr)

    # Solo guardar imagen con rectángulos si no es PDF
    if not is_pdf:
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(output_dir_mtcnn / img_path.name), img_bgr)
    
    print(f"MTCNN processed: {img_path.name}, faces detected: {len(results)}")

In [ ]:
# Process the images
image_counter = 0
max_images = 25

In [ ]:
for folder in input_dir.iterdir():
    if not folder.is_dir():
        continue
    for img_path in folder.glob("*.jpg"):
        if image_counter >= max_images:
            break
        detect_faces_mtcnn(img_path)
        image_counter += 1
    if image_counter >= max_images:
        break

# for folder in input_dir.iterdir():
#     if not folder.is_dir():
#         continue
#     for img_path in folder.glob("*.jpg"):
#         detect_faces_mtcnn(img_path)


In [ ]:
# Show first 5 images without face
print(f"\nTotal images without faces detected (MTCNN): {len(no_faces_detected_mtcnn)}")

for img_path in no_faces_detected_mtcnn[:5]:
    img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f"No face detected: {img_path.name}")
    plt.axis('off')
    plt.show()

---

In [ ]:
# Directory containing PDF files
pdf_dir = Path('pdfs')
output_img_dir = Path('pdf_images_MTCNN')
output_img_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Convert each page of a PDF to a list of images.
def pdf_to_images(pdf_path, output_dir, dpi=300):
    pdf_document = fitz.open(str(pdf_path))
    img_paths = []
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        # 72 es el DPI nativo de PDF
        zoom = dpi / 72  
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat)
        
        img_path = output_dir / f"{pdf_path.stem}_page_{page_num+1}.png"
        pix.save(str(img_path))
        img_paths.append(img_path)
    
    return img_paths

In [ ]:
pdf_files = list(pdf_dir.glob("*.pdf"))

for pdf_file in pdf_files:
    print(f"Processing PDF: {pdf_file.name}")
    try:
        img_paths = pdf_to_images(pdf_file, output_img_dir)
        for img_path in img_paths:
            detect_faces_mtcnn(img_path)
    except Exception as e:
        print(f"Error {pdf_file.name}: {str(e)}")

In [ ]:
def process_pdfs_mtcnn():
    pdf_dir = Path('pdfs')
    pdf_cropped_dir = Path('pdf_cropped_faces_MTCNN')
    pdf_cropped_dir.mkdir(parents=True, exist_ok=True)
    
    pdf_files = list(pdf_dir.glob("*.pdf"))
    
    for pdf_file in pdf_files:
        print(f"\nProcessing PDF: {pdf_file.name}")
        try:
            # Procesar cada página del PDF
            pdf_document = fitz.open(str(pdf_file))
            
            for page_num in range(len(pdf_document)):
                page = pdf_document.load_page(page_num)
                # 300 DPI
                zoom = 300 / 72  
                mat = fitz.Matrix(zoom, zoom)
                pix = page.get_pixmap(matrix=mat)
                
                # Convertir a formato OpenCV
                img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
                img = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Detectar caras
                results = detector.detect_faces(img_rgb)
                
                if len(results) == 0:
                    print(f"No faces detected in page {page_num+1}")
                    continue
                
                # Guardar cada cara recortada
                for idx, res in enumerate(results, start=1):
                    x, y, width, height = res['box']
                    x, y = max(0, x), max(0, y)
                    
                    face_crop = img_rgb[y:y+height, x:x+width]
                    crop_name = f"{pdf_file.stem}_page_{page_num+1}_face_{idx}.jpg"
                    face_bgr = cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(str(pdf_cropped_dir / crop_name), face_bgr)
                    print(f"Saved: {crop_name}")
                    
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")

In [ ]:
# Procesar PDFs
process_pdfs_mtcnn()

---

# MTCNN

In [1]:
# Import necessary libraries
import os
import time
from pathlib import Path
from typing import List, Tuple, Optional, Dict, Any

# Images
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt

# PDF's
import fitz

In [2]:
# Face detection using MTCNN with precision, efficiency, and portability considerations.
class FaceDetector:
    def __init__(self):
        # Configuration
        self.input_dir = Path("photo/images")
        self.output_dir_mtcnn = Path("face_detected_MTCNN")
        self.cropped_dir_mtcnn = Path("cropped_face_MTCNN")
        self.no_face_detected:  List[Path] = []

        # Create directories if they don't exist
        self.output_dir_mtcnn.mkdir(parents=True, exist_ok=True)
        self.cropped_dir_mtcnn.mkdir(parents=True, exist_ok=True)

        # MTCNN configuration
        # self.detector = MTCNN()
        self.detector = MTCNN()


    # # MTCNN detector with optimized parameters.
    # def _initialize_detector(self) -> MTCNN:
    #     return MTCNN(
    #         # Smaller faces more likely to be detected
    #         min_face_size = 20,
    #         # Balance between precision and efficiency
    #         scale_factor = 0.0709,
    #         # Detection thresholds
    #         thresholds = [0.6, 0.7, 0.7],
    #         # Improve detection accuracy
    #         # post_process = True,
    #     )
    

    # Detect faces in an image using MTCNN classifier.
    def detect_faces_MTCNN(self, img_path: Path, is_pdf: bool = False) -> Optional[int]:
        """
        Args:
            img_path: Path of the image file
            is_pdf: Whether the image comes from a PDF (affects ouput). Defaults to False.

        Returns:
            Number of faces detected or None if error ocurred
        """
        # time.perf_counter()
        start_time = time.time()

        try:
            # Read image with error handling
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"Error: Could not read image {img_path}")
                return None
            
            # Convert to RGB
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Detect faces with timing
            detection_time = time.time()
            results = self.detector.detect_faces(img_rgb)
            detection_duration = time.time() - detection_time

            if not results:
                self.no_faces_detected_mtcnn.append(img_path)

            else:
                self._process_detections(img_path, img_rgb, results, is_pdf)

            elapsed = time.time() - start_time
            print(f"MTCNN: {img_path.name} | Faces: {len(results)} | "
                    f"Detection: {detection_duration:.3f}s | Total: {elapsed:.3f}s")
            
            return len(results)
                    
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
            return None
        
    # Process and save detected faces
    def _process_detections(self, img_path: Path, img_rgb: np.ndarray, results: List[Dict[str, Any]], is_pdf: bool) -> None:
        for idx, res in enumerate(results, start=1):
            x, y, w, h = res['box']

            # Coordinates
            x, y = max(0, x), max(0, y)
            w = min(w, img_rgb.shape[1] - x)
            h = min(h, img_rgb.shape[0] - y)

            # Draw rectangle
            if not is_pdf:
                cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Save cropped face
            face_crop = img_rgb[y : y + h, x: x + w]
            self._save_face(img_path, idx, face_crop)

        # Save output image
        if not is_pdf:
            output_path = self.output_dir_mtcnn / img_path.name
            cv2.imwrite(str(output_path), cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

    # Save cropped face with name
    def _save_face(self, img_path: Path, idx: int, face_crop: np.ndarray) -> None:
        crop_name = f"{img_path.stem}_face_{idx}.jpg"
        output_path = self.cropped_dir_mtcnn / crop_name
        # cv2.imwrite(str(self.cropped_dir_mtcnn / crop_name),
        #             cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR))
        cv2.imwrite(str(output_path), cv2.cvtColor(face_crop, cv2.COLOR_RGB2BGR))

    # Process images from input directory with progress tracking
    def process_images(self, max_images: int = 25) -> float:
        image_counter = 0
        
        for folder in self.input_dir.iterdir():
            if not folder.is_dir():
                continue
                
            for img_path in folder.glob("*.jpg"):
                if image_counter >= max_images:
                    return
                    
                self.detect_faces_MTCNN(img_path)
                image_counter += 1
    
    # Display sample images where no faces were detected.
    def show_no_face_samples(self, sample_size: int = 5) -> None:
        print(f"\nTotal images without faces detected: {len(self.no_face_detected)}")
        
        for img_path in self.no_face_detected[:sample_size]:
            try:
                img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(8, 6))
                plt.imshow(img)
                plt.title(f"No face detected: {img_path.name}")
                plt.axis('off')
                plt.show()
            except Exception as e:
                print(f"Error displaying {img_path}: {e}")

In [3]:
# Processor for PDF files using MTCNN
class PDFProcessor:

    # Configuration
    def __init__(self, detector: FaceDetector):
        self.pdf_dir = Path('pdfs')
        self.output_img_dir = Path('pdf_images_MTCNN')
        self.pdf_cropped_dir = Path('pdf_cropped_faces_MTCNN')

        detector.cropped_dir_mtcnn = self.pdf_cropped_dir
        self.detector = detector

        # Create directories if they don't exist
        self.output_img_dir.mkdir(parents=True, exist_ok=True)
        self.pdf_cropped_dir.mkdir(parents=True, exist_ok=True)

    # Process PDF files
    def process_PDFs(self) -> float:
        start_time = time.time()
        pdf_files = list(self.pdf_dir.glob("*.pdf"))

        if not pdf_files:
            print("No PDF files found in directory")
            return 0.0
        
        print("\nStarting PDF processing...")
        for pdf_file in pdf_files:
            self._process_pdf(pdf_file)
        
        return time.time() - start_time
    
    # Process a single PDF file
    def _process_pdf(self, pdf_file: Path) -> None:
        print(f"\nProcessing PDF: {pdf_file.name}")
        pdf_start = time.time()

        try:
            pdf_document = fitz.open(str(pdf_file))

            for page_num in range(len(pdf_document)):
                page_start = time.time()
                page = pdf_document.load_page(page_num)

                # Render page to image (300 DPI)
                pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

                # Convert to np array
                img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(
                    (pix.height, pix.width, 3)
                )

                # Process with face detector
                temp_path = self.output_img_dir / f"temp_{pdf_file.stem}_p{page_num}.png"
                cv2.imwrite(str(temp_path), img_array)
                self.detector.detect_faces_MTCNN(temp_path, is_pdf=True)
                
                # Clean up temporary file
                temp_path.unlink(missing_ok=True)
                
                page_time = time.time() - page_start
                print(f"Page {page_num+1} processed in {page_time:.2f}s")
                
        except Exception as e:
            print(f"Error processing {pdf_file.name}: {str(e)}")
        
        pdf_time = time.time() - pdf_start
        print(f"Finished {pdf_file.name} in {pdf_time:.2f} seconds")

In [4]:
# # Main execution
# if __name__ == "__main__":
#     # Initialize face detector with timing
#     print("Initializing face detector...")
#     init_start = time.perf_counter()
#     face_detector = FaceDetector()
#     print(f"Detector initialized in {time.perf_counter() - init_start:.2f} seconds\n")
    
#     # Process images with detailed timing
#     print("\n" + "="*50)
#     print("Starting image processing...")
#     img_start = time.perf_counter()
    
#     # Process maximum 25 images by default
#     face_detector.process_images(max_images=25)
#     processed_images = 25  
    
#     img_elapsed = time.perf_counter() - img_start
#     print("\n" + "="*50)
#     print(f"IMAGE PROCESSING SUMMARY")
#     print(f"Total images processed: {processed_images}")
#     print(f"Images with faces detected: {processed_images - len(face_detector.no_face_detected)}")
#     print(f"Images without faces: {len(face_detector.no_face_detected)}")
#     print(f"Total processing time: {img_elapsed:.2f} seconds")
#     print("="*50 + "\n")
    
#     # Show samples with no faces detected
#     if face_detector.no_face_detected:
#         print(f"Showing {min(3, len(face_detector.no_face_detected))} samples without detected faces...")
#         face_detector.show_no_face_samples(sample_size=3)

#     # Process PDFs with comprehensive timing
#     print("\n" + "="*50)
#     print("Starting PDF processing...")
#     pdf_start = time.perf_counter()
    
#     pdf_processor = PDFProcessor(face_detector)
#     pdf_processor.process_PDFs()
    
#     pdf_elapsed = time.perf_counter() - pdf_start
#     print("\n" + "="*50)
#     print(f"PDF PROCESSING SUMMARY")
#     print(f"Total processing time: {pdf_elapsed:.2f} seconds")
#     print("="*50 + "\n")
    
#     # Final summary
#     print("\n" + "="*50)
#     print("FINAL SUMMARY")
#     print(f"Total execution time: {time.perf_counter() - init_start:.2f} seconds")
#     print(f"Total faces detected: {processed_images - len(face_detector.no_face_detected)}")
#     print("\n" + "="*50)

In [5]:
# Main execution
if __name__ == "__main__":
    # Initialize face detector with timing
    print("Initializing face detector...")
    init_start = time.perf_counter()
    face_detector = FaceDetector()
    init_time = time.perf_counter() - init_start
    print(f"Detector initialized in {init_time:.2f} seconds\n")
    
    # Process images with detailed timing and averages
    print("\n" + "="*50)
    print("Starting image processing...")
    img_start = time.perf_counter()
    
    # Track processing metrics
    image_counter = 0
    total_faces = 0
    total_time = 0.0
    max_images = 25
    
    # Process images with progress tracking
    for folder in face_detector.input_dir.iterdir():
        if not folder.is_dir():
            continue
            
        for img_path in folder.glob("*.jpg"):
            if image_counter >= max_images:
                break
                
            start_time = time.time()
            faces_detected = face_detector.detect_faces_MTCNN(img_path)
            elapsed = time.time() - start_time
            
            total_time += elapsed
            if faces_detected is not None:
                total_faces += faces_detected
            image_counter += 1
            
            # Display running averages
            avg_time = total_time / image_counter
            avg_faces = total_faces / image_counter if image_counter > 0 else 0
            print(f"Progress: {image_counter}/{max_images} | "
                  f"Avg time: {avg_time:.3f}s | "
                  f"Avg faces: {avg_faces:.1f}")
    
    img_elapsed = time.perf_counter() - img_start
    
    # Image processing summary
    print("\n" + "="*50)
    print("IMAGE PROCESSING SUMMARY")
    print(f"Total images processed: {image_counter}")
    print(f"Images with faces detected: {image_counter - len(face_detector.no_face_detected)}")
    print(f"Images without faces: {len(face_detector.no_face_detected)}")
    print(f"Total faces detected: {total_faces}")
    print(f"Total processing time: {img_elapsed:.2f} seconds")
    print(f"Average time per image: {total_time/image_counter:.3f} seconds")
    print(f"Average faces per image: {total_faces/image_counter:.1f}")
    print("="*50 + "\n")
    
    # Show samples with no faces detected
    if face_detector.no_face_detected:
        print(f"Showing {min(3, len(face_detector.no_face_detected))} samples without detected faces...")
        face_detector.show_no_face_samples(sample_size=3)

    # Process PDFs with comprehensive timing
    print("\n" + "="*50)
    print("Starting PDF processing...")
    pdf_start = time.perf_counter()
    
    pdf_processor = PDFProcessor(face_detector)
    pdf_time = pdf_processor.process_PDFs()
    pdf_elapsed = time.perf_counter() - pdf_start
    
    print("\n" + "="*50)
    print("PDF PROCESSING SUMMARY")
    print(f"Total processing time: {pdf_elapsed:.2f} seconds")
    if pdf_time > 0:  # Now safe because we always return a float
        print(f"PDF-only processing time: {pdf_time:.2f} seconds")
    print("="*50 + "\n")
    
    # Final summary
    print("\n" + "="*50)
    print("FINAL SUMMARY")
    print(f"Total execution time: {time.perf_counter() - init_start:.2f} seconds")
    print(f"Total images processed: {image_counter}")
    print(f"Total faces detected: {total_faces}")
    print(f"Average time per image: {total_time/image_counter:.3f} seconds")
    print(f"Average faces per image: {total_faces/image_counter:.3f}")
    print("="*50)

Initializing face detector...
Detector initialized in 0.22 seconds


Starting image processing...
MTCNN: 00.jpg | Faces: 1 | Detection: 1.946s | Total: 2.189s
Progress: 1/25 | Avg time: 2.192s | Avg faces: 1.0
MTCNN: 01.jpg | Faces: 1 | Detection: 1.598s | Total: 1.864s
Progress: 2/25 | Avg time: 2.029s | Avg faces: 1.0
Error processing photo\images\alb_id\02.jpg: 'FaceDetector' object has no attribute 'no_faces_detected_mtcnn'
Progress: 3/25 | Avg time: 1.932s | Avg faces: 0.7
Error processing photo\images\alb_id\03.jpg: 'FaceDetector' object has no attribute 'no_faces_detected_mtcnn'
Progress: 4/25 | Avg time: 1.877s | Avg faces: 0.5
MTCNN: 04.jpg | Faces: 1 | Detection: 1.625s | Total: 1.873s
Progress: 5/25 | Avg time: 1.877s | Avg faces: 0.6
MTCNN: 05.jpg | Faces: 2 | Detection: 1.613s | Total: 1.857s
Progress: 6/25 | Avg time: 1.874s | Avg faces: 0.8
MTCNN: 06.jpg | Faces: 2 | Detection: 1.761s | Total: 2.003s
Progress: 7/25 | Avg time: 1.893s | Avg faces: 1.0
MTCNN: 07.jpg | Face